In [41]:
import os
import pandas as pd
import re
import random

In [49]:
r1_hs_seqs_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20231212_final_analysis_r1_order/20231221_final_order/20231221_all_design_sets_r1_order.csv'
r2_ssd_seqs_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/ssd/ssd_designs_passing.csv'
r2_msd_seqs_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd/msd_designs_passing.csv'
r2_msd_exp_bbs_seqs_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd_exp_bbs/exp_bbs_msd_designs_passing.csv'
r2_msd_scale_up_seqs_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd_f2s_os_scaled_up/outputs/msd_designs_passing.csv'

outdir = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/final_order'

Single state designs

In [ ]:
r2_ssd_df = pd.read_csv(r2_ssd_seqs_path)
print(f'R2 SSD designs: {r2_ssd_df.shape[0]}')

r2_ssd_df['design_class'] = r2_ssd_df['state_design'] + '-' + r2_ssd_df['seq_method']
#make a shortname column that is the design class plus a number that increases for each row
r2_ssd_df['shortname'] = r2_ssd_df['design_class'] + '-' + r2_ssd_df.index.astype(str)

#get shortname and seqeunce column only
r2_ssd_short_df = r2_ssd_df[['shortname', 'sequence']]

R2 SSD designs: 495


Multistate designs
#load files
#add seq design type column
#give shortened name
#concat dfs

In [ ]:
####MSD for designable bbs####
r2_msd_df = pd.read_csv(r2_msd_seqs_path)
print(f'R2 MSD for designable bbs designs: {r2_msd_df.sequence.nunique()}')

#make a shortname column that is the design class plus a number that increases for each row
r2_msd_df['design_class'] = r2_msd_df['state_design_min0'] + '-' + r2_msd_df['seq_design_method_min0'].str.lower()
r2_msd_df['shortname'] = r2_msd_df['design_class'] + '-' + r2_msd_df.index.astype(str)

#make new column for bb id pair
r2_msd_df['msd_bbs_id'] = r2_msd_df['min0_bb_id_min0'] + r2_msd_df['min2_bb_id_min0']
print(f"Unique bb pairs for R2 MSD F2S scaled up designs: {r2_msd_df['msd_bbs_id'].nunique()}") #may be an overestimate bc of diff naming conventions
r2_msd_short_df = r2_msd_df[['shortname', 'sequence']]

####MSD for experimental bbs####
r2_msd_exp_bbs_df = pd.read_csv(r2_msd_exp_bbs_seqs_path)
print(f'R2 MSD for experimental bbs designs: {r2_msd_exp_bbs_df.sequence.nunique()}')   

#make a shortname column that is the design class plus a number that increases for each row
r2_msd_exp_bbs_df['design_class'] = r2_msd_exp_bbs_df['state_design_min0'] + '-' + r2_msd_exp_bbs_df['seq_design_method_min0'].str.lower()
r2_msd_exp_bbs_df['shortname'] = r2_msd_exp_bbs_df['design_class'] + '-' + r2_msd_exp_bbs_df.index.astype(str)
print(f'Unique bb pairs for R2 MSD exp bbs designs: 1')
r2_msd_exp_bbs_short_df = r2_msd_exp_bbs_df[['shortname', 'sequence']]

####MSD F2S scaled up####
r2_msd_scale_up_df = pd.read_csv(r2_msd_scale_up_seqs_path)
print(f'R2 MSD for F2S scaled up designs: {r2_msd_scale_up_df.sequence.nunique()}')

#make a shortname column that is the design class plus a number that increases for each row
r2_msd_scale_up_df['design_class'] = r2_msd_scale_up_df['state_design_min0'] + '-' + r2_msd_scale_up_df['seq_design_method_min0'].str.lower()
r2_msd_scale_up_df['shortname'] = r2_msd_scale_up_df['design_class'] + '-' + r2_msd_scale_up_df.index.astype(str)
print(f"Unique bb pairs for R2 MSD F2S scaled up designs: {r2_msd_scale_up_df['msd_bbs_id_min0'].nunique()}")
r2_msd_scale_up_short_df = r2_msd_scale_up_df[['shortname', 'sequence']]


R2 MSD for designable bbs designs: 121
Unique bb pairs for R2 MSD F2S scaled up designs: 65
R2 MSD for experimental bbs designs: 153
Unique bb pairs for R2 MSD exp bbs designs: 1
R2 MSD for F2S scaled up designs: 326
Unique bb pairs for R2 MSD F2S scaled up designs: 58


Multistate predicted as only one state

In [ ]:
#for ones predicted in only one state filter more stringently
#sample 15% from every set
#add seq design type column
#give shortened name
#concat dfs

#take MSD sequences that are low RMSD to only one state, refilter for plddt > 90 and see how many
r2_msd_min0_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd/msd_designs_passing_min0_only.csv'
r2_msd_min2_path = '/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd/msd_designs_passing_min2_only.csv'

r2_msd_min0_df = pd.read_csv(r2_msd_min0_path)
r2_msd_min0_df = r2_msd_min0_df.query('avg_plddt_no_loop > 90  & min0_all_rmsd_no_loop < 1.0 & avg_pae_no_loop < 5', engine='python').copy()
print(f'Number of MSD designs predicted to min0 passing stringent filters: {r2_msd_min0_df.sequence.nunique()}')
r2_msd_min0_df_subset = r2_msd_min0_df.sample(n=250, axis=0, random_state=42)


#make design class and shortname columns


r2_msd_min2_df = pd.read_csv(r2_msd_min2_path)
r2_msd_min2_df = r2_msd_min2_df.query('avg_plddt_no_loop > 90  & min2_all_rmsd_no_loop < 1.0  & avg_pae_no_loop < 5', engine='python').copy()
print(f'Number of MSD designs predicted to min2 passing stringent filters: {r2_msd_min2_df.sequence.nunique()}')
r2_msd_min2_df_subset = r2_msd_min2_df.sample(n=250, axis=0, random_state=42)


Number of MSD designs predicted to min0 passing stringent filters: 9446
Number of MSD designs predicted to min2 passing stringent filters: 572
(250, 40) (250, 40)


In [ ]:
#concat all ssd and msd and calculate max pairwise seq identity

Control sequences

In [ ]:
#make df with additional control seqs from Bram and 375 and point mutants
#TODO: add alfa tag mutants with bm01 ctrl, add more conservative ad to ala
bespoke_ctrls_dict = {'bmp38' : 'GARLAQLKQERAALKQRLAALDQEIAALEWQIQSDPRKKQLYQRLLALISDRFALEQRIAALDQEIAALEAG', 
                 'bmp53' : 'GARLEELIAERLRLVGDLVDLDREIAALEQQIQSDPRKKQLEQRLAALKQERAALEQRIAALDWEIADLDDG', 
                 'bmp31' : 'GARLAQLKQERAALKQRLAALDQEIAALEWQIQSDPRKKQLEQRLAALKQERAALEQRIAALDQEIAALEAG', 
                 'bmp36' : 'GARLAQLKQERAALEQRLAALDQEIAALEWQIQSDPRKKQLEQKLLELQAERLRLIGDIVNLDQQILNLEAG', 
                 'bmp42' : 'GARLAQLKQERAALKQRLDALDQEIAALEWQIQSDPRKKQLYQRLLELIGERLALMGDIFELDVEIAALEAG', 
                 'bmp45' : 'GARLAQLKQERAALKQRLEALDQEIAALEWQIQSDPRKKQLLQRLYELFGERLALFGDIFDLDVEIAALEAG', 
                 'bmp51' : 'GARLAQLKQERAALKQRLDALEQEIAALEWQIQSDPRKKQLQQRLRQLYGERLRLESDIFDLDVEIAALEAG', 
                 'bmp57' : 'GARLAQLKQERAALKQRLEALDQEIAALEWQIQSDPRKKQLLQRLRRLYGERLALFGDIFDLDVEIAALEAG', 
                 'bmp23' : 'GARLAQLKQERAALKQRLAALDQEIAALEWQIQSPDRKKQLEQRLAALKQERAALEQRIAALDQEIAALEAG',
                 '375' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-R74W' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRWLE',
                 '375-L75W' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRWE',
                 '375-77W' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLEW',
                 '375-QatF' : 'QELKRIEQEIAAIEQEIARAEQKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLEW',
                 '375-QatF-cdel' : 'QELKRIEQEIAAIEQEIARAEQKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTE',
                 '375-ad-to-A' : 'EEAKRAEEEAAAAEREAARAEEKAKAQESDPRKKGLQAEAEKALEKAAEARKEAERASRLEEELRRRLTELRRRLE',
                 '375-ad-to-A-cdel' : 'EEAKRAEEEAAAAEREAARAEEKAKAQESDPRKKGLQAEAEKALEKAAEARKEAERASRLEEELRRRLTE', 
                 '375-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTE', 
                 '375-G53Q' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKQLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE', 
                 '375-G35Q-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKQLQAEREKLLEKLAELRKERERLSRLEEELRRRLTE', 
                 '375-E14Q' : 'EELKRIEEEIAAIQREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE', 
                 '375-E14Q-cdel' : 'EELKRIEEEIAAIQREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRLEEELRRRLTE', 
                 '375-E14Q-G35Q' : 'EELKRIEEEIAAIQREIARAEEKLKAQESDPRKKQLQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-E14Q-G35Q-cdel' : 'EELKRIEEEIAAIQREIARAEEKLKAQESDPRKKQLQAEREKLLEKLAELRKERERLSRLEEELRRRLTE',
                 '375-G35A' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKALQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE', 
                 '375-G35A-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKALQAEREKLLEKLAELRKERERLSRLEEELRRRLTE', 
                 '375-L39P' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGPQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-L39P-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGPQAEREKLLEKLAELRKERERLSRLEEELRRRLTE',
                 '375-L36K' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGKQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-L36K-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGKQAEREKLLEKLAELRKERERLSRLEEELRRRLTE',
                 '375-L36Q' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGQQAEREKLLEKLAELRKERERLSRLEEELRRRLTELRRRLE',
                 '375-L36Q-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGQQAEREKLLEKLAELRKERERLSRLEEELRRRLTE',
                 '375-ALFA-LtoA' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRAEEEARRRATELRRRLE',
                 '375-ALFA-LtoA-cdel' : 'EELKRIEEEIAAIEREIARAEEKLKAQESDPRKKGLQAEREKLLEKLAELRKERERLSRAEEEARRRATE'}

#make df with columns 'Name and 'sequence' from dict
bespoke_ctrls_df = pd.DataFrame.from_dict(bespoke_ctrls_dict, orient='index', columns=['sequence']).reset_index().rename(columns={'index': 'Name'})
bespoke_ctrls_df.shape

(34, 2)

In [ ]:
#control seq functions

####Heptad a and d to Ala####
def heptad_to_upper(input_string, split_char = 'r '):
    split_1 = input_string.split(split_char)[0]
    split_2 = input_string.split(split_char)[1]
    split_2 = split_2.upper()
    
    new_string = split_1+split_char+split_2

    return new_string


def heptad_a_d_to_ala(reg_string, seq_string):

    og_seq = re.split(r'\s+', seq_string)[1]
    
    for index, char in enumerate(reg_string):
        if char.isupper():
            if char in ['A', 'D']:
                seq_string = seq_string[:index] + 'A' + seq_string[index+1:]
            else:
                continue
    
    new_seq = re.split(r'\s+', seq_string)[1]

    return og_seq, new_seq


def a_d_to_ala(input_df):

    new_seqs_dict = {}

    for index, row in input_df.iterrows():
        design_id = row['design_id']

        h1_reg = row['h1_reg']
        new_h1_reg = heptad_to_upper(h1_reg)

        h1_seq = row['h1_seq']

        h1_og, h1_new = heptad_a_d_to_ala(new_h1_reg, h1_seq)

        h2_reg = row['h2_reg']
        new_h2_reg = heptad_to_upper(h2_reg)

        h2_seq = row['h2_seq']

        h2_og, h2_new = heptad_a_d_to_ala(new_h2_reg, h2_seq)

        #replace with ala seq in sequence
        sequence = row['sequence']
        new_sequence = sequence.replace(h1_og, h1_new)
        new_sequence = new_sequence.replace(h2_og, h2_new)

        new_seqs_dict[design_id] = new_sequence

    new_df = input_df.copy()
    new_df['a_d_to_ala_sequence'] = new_df['design_id'].map(new_seqs_dict)

    return new_df

####

####SCRAMBLE CONTROLS####
def full_scrambler(sequence_list):
    '''function for full scrambles use random.sample
    Takes a list of original design sequences
    Outputs a dictionary with original seq and fully scrambled seq as key:value pair'''

    full_scramble_dict = {}
    for seq in sequence_list:
        residues = list(seq)
        sc_residues = random.sample(residues, k=len(residues))
        sc_seq = ''.join(sc_residues)
        full_scramble_dict[seq] = sc_seq
    return full_scramble_dict

def patterned_scrambler(sequence_list):
    '''Function for creating patterned scrambled designs
    In which hydrophobic or polar residues are shuffled to different positions with the same h or p character
    prolines and glycines are kept in original positions
    based on criteria described in in Rocklin, et al. 2017
    Takes a list of original design sequences
    Outputs a data frame with the original sequence, pattern of H/P/Pro/Gly, and the patterned scramble sequence'''

    patterned_scramble_dict = {}
    #Rocklin term these polar if they occur in a helix: D, E, H, K, N, Q, R, S, T, or Y
    #A, F, I, L, M, V, W, and Y used to count number of hydrophobic aa
    #I just used hydrophobic and polar definitions from default resfile categories
    hydrophobics_list = ['A', 'F', 'I', 'L', 'M', 'V', 'W', 'Y']
    polar_list = ['D', 'E', 'H', 'K', 'N', 'Q', 'R', 'S', 'T']
    seq_pattern_dict = {}

    for seq in sequence_list:
        residues = list(seq)
        patterned_res = []
        hydrophobic_residues = []
        polar_residues = []
        patterned_scramble_res = []
        for res in residues:
            if res == 'P':
                patterned_res.append('Pro')

            elif res == 'G':
                patterned_res.append('G')

            elif res in hydrophobics_list:
                patterned_res.append('H')
                hydrophobic_residues.append(res)

            elif res in polar_list:
                patterned_res.append('P')
                polar_residues.append(res)

            else:
                print('Cys found!')

        seq_pattern = ''.join(patterned_res)
        seq_pattern_dict[seq] = seq_pattern

        for res in patterned_res:

            if res == 'Pro':
                patterned_scramble_res.append('P')
            elif res == 'G':
                patterned_scramble_res.append('G')
            elif res == 'H':
                l = random.randint(0, len(hydrophobic_residues)-1)
                r = hydrophobic_residues.pop(l)
                patterned_scramble_res.append(r)
            elif res == 'P':
                l = random.randint(0, len(polar_residues)-1)
                r = polar_residues.pop(l)
                patterned_scramble_res.append(r)
        pattern_scramble_seq = ''.join(patterned_scramble_res)
        patterned_scramble_dict[seq] = pattern_scramble_seq

    df_name_pattern = pd.DataFrame.from_dict(seq_pattern_dict, orient='index', columns=['Seq HP pattern'])
    df_name_pattern.index.name = 'Insert protein sequence'
    df_name_pattern.reset_index(inplace=True)

    df_name_pattern_sc = pd.DataFrame.from_dict(patterned_scramble_dict, orient='index', columns=['Patterned scramble insert protein sequence'])
    df_name_pattern_sc.index.name = 'Insert protein sequence'
    df_name_pattern_sc.reset_index(inplace=True)

    df = pd.merge(df_name_pattern, df_name_pattern_sc, on='Insert protein sequence')

    return df

####

#200 patterned scramble
#200 full scramble
#200 a and d to Ala 
#split 50 each MPNN and F2s and SSD and MSD

In [40]:
test_heptad = 'register       abcdefgabcdefgabcdefgabcd          '
test_str = 'sequence PLREKLLELKERLKELLELKLEELSRLEEELRRRLTEEERR'

import re
new_heptad = heptad_to_upper(test_heptad)
print(new_heptad)


test_dict = heptad_a_d_to_ala(new_heptad, test_str)
print(test_dict)

#need to exclude alfa tag: onyl change a and d for min2 state and don't need to worry about this

test_df = pd.read_csv('/Users/stephaniecrilly/Library/CloudStorage/Box-Box/kortemmelab/home/scrilly/helix_sliding/20250604_r2_hs_lib/metric_files/msd/msd_designs_passing_min2_only.csv')

new_test_df = a_d_to_ala(test_df)
new_test_df.to_csv('/Users/stephaniecrilly/Downloads/test_a_d_to_ala.csv', index=False)

register       ABCDEFGABCDEFGABCDEFGABCD          
('PLREKLLELKERLKELLELKLEELSRLEEELRRRLTEEERR', 'PLREKLAELAERLAELAELKAEEASRLAEEARRRLTEEERR')


In [ ]:
#generate post alfa truncated seqs from r1_hs_seqs and all others
#then remove any of duplciates already in r2_ssd (should be 64)

#add in nucleotide sequences for round 1 designs at the very end